In [3]:
!ls ./data

 N02-23_RailroadSection.geojson   N06-23_Joint.dbf
 N02-23_Station.geojson		  N06-23_Joint.geojson
 N06-23.xml			  N06-23_Joint.prj
 N06-23_HighwaySection.dbf	  N06-23_Joint.shp
 N06-23_HighwaySection.geojson	  N06-23_Joint.shx
 N06-23_HighwaySection.prj	  meizan.geojson
 N06-23_HighwaySection.shp	 '国土数値情報 _ 鉄道データ.pdf'
 N06-23_HighwaySection.shx	 '国土数値情報 _ 高速道路時系列データ.pdf'


In [138]:
import numpy as np
import geopandas as gpd
import pandas as pd
import json
import geojson

'''
    高速道路のデータの処理
    shp -> geojson
    IDは路線名ごとにつけている
'''

def main():

    ifile = "./data/N06-23_HighwaySection.shp"
    gdf = gpd.read_file(ifile, encoding='utf-8')
    df_name = gdf.dissolve(by="N06_007", as_index=False)["N06_007"]

    ## 路線名ごとにidをふる
    names = df_name.to_list()
    ids = np.arange(len(names))
    name_dict = {key: val for key, val in zip(names, ids)}

    features = []
    for i in range(len(gdf)):
    #for i in range(1):
        lons = np.array(gdf["geometry"][i].xy[0])
        lats = np.array(gdf["geometry"][i].xy[1])
        lnglat = [xy for xy in zip(lons, lats)]
        name = gdf["N06_007"][i]
        id = name_dict[name]
        feature = {
            "type": "Feature",
            #"id": str(gdf["id"][i]),
            "id": str(id),
            "properties": {
                "供用開始年": str(gdf['N06_001'][i]),
                "開始年": str(gdf['N06_002'][i]),
                "終了年": str(gdf['N06_003'][i]),
                "関係ID": str(gdf['N06_004'][i]),
                "変遷ID": str(gdf['N06_005'][i]),
                "変遷備考": str(gdf['N06_006'][i]),
                "路線名": str(gdf['N06_007'][i]),
                "路線種別": str(gdf['N06_008'][i]),
                "併用状況": str(gdf['N06_009'][i]),
                "車線数": str(gdf['N06_010'][i]),
                "備考": str(gdf['N06_011'][i]),
            },
            "geometry": {
                "type": "LineString",
                "coordinates": lnglat,
            },
        }
        features.append(feature)

    # GeoJSONファイルを書き込む
    ofile = "./data/highway.geojson"
    with open(ofile, "w", encoding='utf_8') as f:
        json.dump({"type":"FeatureCollection", "features": features}, f, indent = 2, ensure_ascii=False)


if __name__ == "__main__":
    main()